In [2]:
import pandas as pd
import numpy as np
import psycopg2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
df = pd.read_csv("scheme_data.csv")

In [5]:
df['vrow'] =df['vcol'] = df['vectorembeddings'] = ['0' for i in range(len(df['Exclusion']))]

In [10]:
df['SchemeID'] = [str(i) for i in range(len(df['Exclusion']))]

In [14]:
df = df.fillna('0')
df = df.replace(0,'0')

In [15]:
vectorizer = TfidfVectorizer(stop_words='english')
for i in range(len(df['DocumentsReq'])):
    content = df.iloc[i][:-4]
    vectors = vectorizer.fit_transform(content)
    vec = vectors.A
    col = [len(vec[0])]
    row = [len(vec)]
    embeddings = [0]
    for k in range(len(vec)):
        for j in range((k)):
            if vec[k][j]!=0:
                row.append(k)
                col.append(j)
                embeddings.append(vec[k][j])
    df['vrow'][i] = row
    df['vcol'][i] = col
    df['vectorembeddings'][i] = embeddings
        

In [16]:
df

,SchemeName,Details,Benefits,Eligibility,Exclusion,ApplicationProc,DocumentsReq,FAQ,Link,Keywords,vrow,vcol,vectorembeddings,SchemeID
0,ICMR- Post Doctoral Fellowship,DetailsICMR- Post Doctoral Fellowship (ICMR-PD...,BenefitsICMR-PDFs will be paid a consolidated ...,EligibilityICMR-PDF is open to Indian national...,0,Application ProcessOfflineLast Date of Applica...,Documents Required1.\tPassport-size Photograph...,Frequently Asked QuestionsWhat is the “ICMR- P...,https://www.myscheme.gov.in/schemes/icmr-pdf,FellowshipICMRPDFPost DoctoralResearch,"[10, 2, 2, 3, 5, 5, 5, 7, 7]","[355, 0, 1, 2, 2, 3, 4, 0, 1]","[0, 0.09214978214962381, 0.18429956429924763, ...",0
1,Dr. Ambedakar Centrally Sponsored Scheme of Po...,"Details""Dr. Ambedakar Centrally Sponsored Sche...",BenefitsMaintenance Allowance﻿Group ADegree an...,EligibilityThe scholarships will be open to In...,ExclusionsScholarships shall not be awarded fo...,Application ProcessOnlineStep 1: Visit the N...,Documents RequiredOne copy of the application ...,Frequently Asked QuestionsIs the scholarship p...,https://www.myscheme.gov.in/schemes/dacsspostm...,EBCEconomically Backward ClassPost MatricSchol...,"[10, 3, 3, 7]","[519, 0, 1, 3]","[0, 0.10721368995173382, 0.10721368995173382, ...",1
2,Savitribai Jyotirao Phule Fellowship For Singl...,DetailsSavitribai Jyotirao Phule Fellowship fo...,BenefitsTENURE OF FELLOWSHIP:﻿The tenure of th...,Eligibility(i) Any single girl child of her pa...,"ExclusionsIf in a family, there is one or more...",Application ProcessOnlineApplications are invi...,Documents RequiredScanned copy of your passpor...,Frequently Asked QuestionsDoes Only The Submis...,https://www.myscheme.gov.in/schemes/sjpfsgc,FellowshipGirl ChildPh.D.,"[10, 2, 5, 5, 6, 7, 7]","[487, 0, 2, 3, 3, 1, 4]","[0, 0.1938574018107867, 0.030486749196754388, ...",2
3,Scheme of Assistance for the Prevention of Alc...,"Details""Scheme of Assistance for the Preventio...",BenefitsItems Admissible for AssistanceConstru...,EligibilityFor Organizations/ InstitutionsThe ...,0,Application ProcessOnlineStep 1: Application S...,Documents RequiredFor Building ConstructionSub...,Frequently Asked QuestionsHow does the continu...,https://www.myscheme.gov.in/schemes/sapaandsda...,Financial AssistanceGrantRehabilitationSocial ...,"[10, 2, 2, 7, 7, 7, 7, 7]","[417, 0, 1, 0, 1, 3, 4, 6]","[0, 0.11143681373595711, 0.33431044120787134, ...",3
4,Support for Value Addition - Support to R&G units,"DetailsThe scheme ""Support for Value Addition ...","BenefitsRoasting Units, Gourmet roasting units...",EligibilityThe applicants should be individual...,ExclusionsThe applicants who have availed subs...,Application ProcessOfflineStep 1: The interest...,Documents RequiredCopy of the blueprint/layout...,Frequently Asked QuestionsIs there a subsidy l...,https://www.myscheme.gov.in/schemes/icdpsva-srgu,AgricultureDirect Benefit TransferFarmerFinanc...,"[10, 2, 2, 4, 4, 7, 7]","[363, 0, 1, 2, 3, 4, 5]","[0, 0.2084740158001447, 0.2084740158001447, 0....",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2402,Mukhya Mantri Mahila Sashaktikaran Yojana,"DetailsThe scheme ""Mukhya Mantri Mahila Sashak...","BenefitsUnder this scheme, permanent training ...",EligibilityThe applicant should be a rape vict...,0,Application ProcessOfflineThe application will...,Documents RequiredApplicant’s Photo ID ProofAa...,Frequently Asked QuestionsWhen was the scheme ...,https://www.myscheme.gov.in/schemes/mmmsy,EmploymentMahilaSashaktikaranTrainingWoman,"[10, 1, 2, 3, 6, 7, 7, 7, 7]","[182, 0, 1, 2, 4, 0, 2, 3, 5]","[0, 0.06775881319435155, 0.13862774488753984, ...",2402
2403,IMA Nongthangleima Yaipha Tengbang Scheme,"DetailsThe ""Ima Nongthangleima Yaipha Tengbang...",BenefitsFinancial Assistance of ₹500/- per month.,EligibilityThe applicant should be a woman. Th...,0,Application ProcessOfflineApplication Process:...,Documents RequiredPassport-size Photo.Aadhar C...,Frequently Asked QuestionsWho is eligible for ...,https://www

In [17]:
df.to_csv("VectoredSchemes.csv")

In [18]:
import psycopg2
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_recommendations(user_keywords):
    """Returns top 5 schemes based on user provided keywords."""
    schemes = pd.read_csv("VectoredSchemes.csv")
    if schemes is not None:
        scheme_data = [' '.join(map(str, scheme)) for scheme in schemes]

        # Vectorize scheme data and user input using TF-IDF
        vectorizer = TfidfVectorizer(stop_words='english')
        vectors = vectorizer.fit_transform(scheme_data + [user_keywords])

        # Calculate cosine similarity
        cosine_sim = cosine_similarity(vectors[-1], vectors[:-1]).flatten()

        # Get indices of top 5 schemes
        top_indices = cosine_sim.argsort()[-5:][::-1]
        top_schemes = [schemes[i] for i in top_indices]

        return top_schemes
    else:
        print("No schemes found in the database.")

def chatbot():
    """Runs the chatbot interface."""
    print("Welcome to the Government Schemes Recommender System!")
    user_input = input("Please enter keywords related to the scheme you are looking for: ")
    recommendations = get_recommendations(user_input)

    if recommendations:
        print("\nTop 5 Schemes:")
        for scheme in recommendations:
            print(f"Scheme Name: {scheme[0]}")
            print(f"Details: {scheme[1]}")
            print(f"Benefits: {scheme[2]}")
            print(f"Eligibility: {scheme[3]}")
            print(f"Exclusion: {scheme[4]}")
            print(f"Application Procedure: {scheme[5]}")
            print(f"Documents Required: {scheme[6]}")
            print(f"FAQ: {scheme[7]}")
            print(f"Link: {scheme[8]}")
            print(f"Keywords: {scheme[9]}\n")
    else:
        print("No recommendations found.")
chatbot()

Welcome to the Government Schemes Recommender System!


Please enter keywords related to the scheme you are looking for:  women scholarship


KeyError: 14